# Easy Naive Bayes model

This Python notebook is about a simple Naive Bayes model implementation applied to The Spam Filtering problem. The dataset used is the <b> SpamAssassin public corpus </b> (https://spamassassin.apache.org/old/publiccorpus/, look for the three files under the prefix 20021010).   

To test the model, after extracting the data you should have three folders: <b> spam</b>, <b>easy_ham</b>, and <b>hard_ham</b>. Each folder contains many emails, each contained in a single file. To keep things really simple, we’ll just look at the subject lines of each email.

## Step 1: tokenization

Let’s create a simple function to tokenize messages into distinct words. We’ll first convert each message to lowercase, then we'll use `re.findall()` to extract “words” (strings) consisting of letters, numbers, and apostrophes. Finally we will use `set()` to get just the distinct words.

In [1]:
def tokenize(message):
    message = message.lower()                     # convert to lowercase
    all_words = re.findall("[a-z0-9']+", message) # extract the words
    
    return set(all_words)                         # remove duplicates

The next function will count the words in a set of labeled messages (the training set) and it will return a dictionary where each key-value pair has the form `(word, [spam_count, non_spam_count])`.

In [2]:
from collections import defaultdict

def count_words(training_set):
    """training set consists of pairs (message, is_spam)"""
    counts = defaultdict(lambda: [0, 0])
    for message, is_spam in training_set:
        for word in tokenize(message): # for each word
            # increase the word count according to 
            # spam/non-spam condition
            counts[word][0 if is_spam else 1] += 1
    
    return counts

### The Naive Bayes assumption

Consider am email message $E$ containing words $w_1,\dots,w_n$. The probability of receiving the email $E$ is equal to the probability of receiving the list of words $w_1, \dots, w_n$

$$\mathrm{Pr}(w_1,\dots,w_n)\,.$$

This probability is impossible to evaluate in practice because if the email contains a large number of words, then one needs a huge training set containing all the possible combinations of the probabilities of words in the email. Even if we had a reasonably large dataset, it would be still very unlikely for us to find all the possible combinations everytime we receive a new email. But we can roughly overcome this issue if we consider the following (very strong) <b>assumption</b>:

$$\mathrm{Pr}(w_1,\dots,w_n) = \mathrm{Pr}(w_1) \cdots \mathrm{Pr}(w_n) \,.$$

This means that the words in the email are <b>independent</b> of each other and randomly displayed in the email. Despite this assumption is clearly <b>false</b> in most real-world cases, Naive Bayes often performs classification very well.

Let's denote two classes of emails as spam $S$ and non-spam $\neg S$. Let $\mathrm{Pr}(E \mid S)$ be the probability that a given email from class $S$ is the email $E$ and let $\mathrm{Pr}(E \mid \neg S)$ the probability that a given non-spam email is $E$. We can express $\mathrm{Pr}(E \mid S)$ as 

$$ \mathrm{Pr}(E \mid S) = \mathrm{Pr}(w_1,\dots,w_n \mid S) = \mathrm{Pr}(w_1 \mid S) \cdots \mathrm{Pr}(w_n \mid S)\,.  $$

This very strong assumption leads to an oversimplified model (for this reason the method is called "naive"). In practice, to avoid underflow (computers don't deal well with numbers too close to zero) we compute

$$
\exp\left[ \log(p_1) + \cdots + \log(p_n) \right]
$$

where $p_i =  \mathrm{Pr}(w_i \mid S)$.

We use Bayes rule to compute the posterior probability of spam email given the overall probability of the sampling email; this is the crucial part of the entire classification:

$$ \mathrm{Pr}(S \mid E) = \frac{\mathrm{Pr}(S) \cdot \mathrm{Pr}(E \mid S) }{\mathrm{Pr}(E)} = \frac{\mathrm{Pr}(S) \cdot \prod_{i}  \mathrm{Pr}(w_i \mid S)}{\mathrm{Pr}(E)}\,.$$

An analogous formula holds for $\mathrm{Pr}(\neg S \mid E)$. Furthermore, using Bayes formula we can write

$$ \mathrm{Pr}(S \mid E) = \frac{ \mathrm{Pr}(S) \mathrm{Pr}(E \mid S)} {\mathrm{Pr}(E)} = \frac{ \mathrm{Pr}(S) \mathrm{Pr}(E \mid S)} {\mathrm{Pr}(S)\mathrm{Pr}(E \mid S) + \mathrm{Pr}(\neg S)\mathrm{Pr}(E \mid \neg S)} \,,$$

and if we further assume that any message is equally likely to be spam or not-spam, so that $\mathrm{Pr}(S) = \mathrm{Pr}(\neg S) = 0.5$, the formula simplifies to

$$
 \mathrm{Pr}(S \mid E) = \frac{ \mathrm{Pr}(S) \mathrm{Pr}(E \mid S)} {\mathrm{Pr}(E)} = \frac{\mathrm{Pr}(E \mid S)} {\mathrm{Pr}(E \mid S) + \mathrm{Pr}(E \mid \neg S)} \,.
$$

We will use the last formula in Step 3.
If we have a fair number of “training” messages labeled as spam and non-spam, an obvious first try is to estimate  $\mathrm{Pr}( w_i \mid S)$ simply as the <b>fraction</b> of spam messages containing the word $w_i$. Intuition is fine but there's a problem: imagine that in our training set the vocabulary word “info” only occurs in nonspam messages. Then we would get $\mathrm{Pr}(\,\mathsf{info} \mid S) = 0$. The result is that our Naive Bayes classifier would always assign spam probability $0$ to any message containing the word “info", even a message like “info about Rolex watches.” To avoid this problem, we usually use some kind of <b>smoothing</b>.

### Smoothing
When computing the spam probabilities for $w_i$, we assume we also saw $k$ additional spams containing the word $w_i$ and $k $additional spams not containing the word. So, choose a count $k$ and estimate the probability of seeing the word $w_i$ in a spam as:

$$\mathrm{Pr}(w_i \mid S) = \frac{ k + \mathsf{number\;of\;spams\;containing\;} w_i}{ 2k + \mathsf{number\;of\;spams}}\,.$$

Analogously for $\mathrm{Pr}(w_i \mid \neg S)$. 

Example: if “info” occurs in 0/48 spam documents, and if $k$ is $1$, we estimate

$$ \mathrm{Pr}\,(\,\mathsf{info} \mid S\,) = \frac{ 1 + 0}{2\cdot1 + 48} = \frac{1}{50} = 0.02 \,,$$

which allows our classifier to still assign some nonzero spam probability to messages containing the word “info".

## Step 2: turn counts into probabilities

Our next function aims to turn these counts into estimated probabilities using the smoothing we described before. The function will return a list of triplets containing each word, the probability of seeing that word in a spam message, and the probability of seeing that word in a nonspam message.

In [3]:
def word_probabilities(counts, total_spams, total_non_spams, k=0.5):
    """turn the word_counts into a list of triplets 
       [w, Pr(w | spam), Pr(w | non-spam)]"""
    
    return [(w,
            (spam + k) / (total_spams + 2 * k),
            (non_spam + k) / (total_non_spams + 2 * k))
            for w, (spam, non_spam) in counts.items()]

## Step 3: assign probabilities to messages

Now we use the evaluated word probabilities to assign probabilities to messages.

In [4]:
import math

def spam_probability(word_probs, message):
    '''returns Pr(S|E)'''
    # tokenize message
    message_words = tokenize(message)
    # initialize log probabilities
    log_prob_if_spam = log_prob_if_not_spam = 0.0
    # iterate through each word in our vocabulary
    for word, prob_if_spam, prob_if_not_spam in word_probs:
        # if *word* appears in the message,
        # add the log probability of seeing it
        if word in message_words:
            log_prob_if_spam += math.log(prob_if_spam)
            log_prob_if_not_spam += math.log(prob_if_not_spam)
        # if *word* doesn't appear in the message
        # add the log probability of _not_ seeing it
        # which is log(1 - probability of seeing it)
        else:
            log_prob_if_spam += math.log(1.0 - prob_if_spam)
            log_prob_if_not_spam += math.log(1.0 - prob_if_not_spam)

    prob_if_spam = math.exp(log_prob_if_spam)
    prob_if_not_spam = math.exp(log_prob_if_not_spam)
    
    return prob_if_spam / (prob_if_spam + prob_if_not_spam)

## Step 4: Naive Bayes classifier

The following is a classifier class encompassing all the function defined so far.

In [5]:
class NaiveBayesClassifier:
    
    def __init__(self, k=0.5):
        self.k = k
        self.word_probs = []
        
    def train(self, training_set):
        # count spam and non-spam messages
        num_spams = len([is_spam
                        for message, is_spam in training_set
                        if is_spam])
        num_non_spams = len(training_set) - num_spams
        
        # run training data through our "pipeline"
        word_counts = count_words(training_set)
        self.word_probs = word_probabilities(word_counts,
                                            num_spams,
                                            num_non_spams,
                                            self.k)
        
    def classify(self, message):
        
        return spam_probability(self.word_probs, message)

## Step 5: removing subject identifier from lines

Files starting with "Subject:" are subject lines.

In [6]:
import glob, re

# define the path containing the files
path = r".\*\*"
data = []

# glob.glob returns every filename that matches the wildcarded path
for fn in glob.glob(path):
    # set as spam if ham is not in the filename
    is_spam = "ham" not in fn
    
    with open(fn, 'r', errors='ignore') as file:
        for line in file:
            if line.startswith("Subject:"):
                # remove the leading "Subject: " and keep the rest of the line
                subject = re.sub(r"^Subject: ", "", line).strip()
                data.append((subject, is_spam))

In [7]:
data[:10]

[('Re: New Sequences Window', False),
 ('[zzzzteana] RE: Alexander', False),
 ('[zzzzteana] Moscow bomber', False),
 ("[IRR] Klez: The Virus That  Won't Die", False),
 ('Re: Insert signature', False),
 ('Re: [zzzzteana] Nothing like mama used to make', False),
 ('Re: [zzzzteana] Nothing like mama used to make', False),
 ('[zzzzteana] Playboy wants to go out with a bang', False),
 ('Re: [zzzzteana] Nothing like mama used to make', False),
 ('[zzzzteana] Meaningful sentences', False)]

## Step 6:  split the data

We split the data into training data and test data.

In [8]:
def split_data(data, prob):
    """split data into fractions [prob, 1 - prob]"""
    results = [], []
    for row in data:
        results[0 if random.random() < prob else 1].append(row)
    
    return results

In [9]:
import random

random.seed(0) 
train_data, test_data = split_data(data, 0.75)

In [10]:
train_data[:5], test_data[:5]

([('[zzzzteana] Moscow bomber', False),
  ("[IRR] Klez: The Virus That  Won't Die", False),
  ('Re: Insert signature', False),
  ('Re: [zzzzteana] Nothing like mama used to make', False),
  ('[zzzzteana] Playboy wants to go out with a bang', False)],
 [('Re: New Sequences Window', False),
  ('[zzzzteana] RE: Alexander', False),
  ('Re: [zzzzteana] Nothing like mama used to make', False),
  ('Re: New Sequences Window', False),
  ('Re: New Sequences Window', False)])

## Step 7: testing the model

Let's build a classifier instance and then start training.

In [11]:
classifier = NaiveBayesClassifier()
classifier.train(train_data)

In [12]:
# triplets (subject, actual is_spam, predicted spam probability)
classified = [(subject, is_spam, classifier.classify(subject))
                for subject, is_spam in test_data]

In [13]:
classified[:10]

[('Re: New Sequences Window', False, 1.7492787736962358e-05),
 ('[zzzzteana] RE: Alexander', False, 4.36722330733585e-05),
 ('Re: [zzzzteana] Nothing like mama used to make',
  False,
  0.0006355823082923374),
 ('Re: New Sequences Window', False, 1.7492787736962358e-05),
 ('Re: New Sequences Window', False, 1.7492787736962358e-05),
 ('[ILUG] Re: Problems with RAID1 on cobalt raq3',
  False,
  0.0014660556345939412),
 ('Re: New Sequences Window', False, 1.7492787736962358e-05),
 ('The case for spam', False, 0.0017751881200852985),
 ("[IIU] Eircom aDSL Nat'ing", False, 0.06839623756971493),
 ('[zzzzteana] Which Muppet Are You?', False, 0.03531988332866546)]

In [14]:
from collections import Counter

# assume that spam_probability > 0.5 corresponds to spam prediction
# and count the combinations of (actual is_spam, predicted is_spam)
counts = Counter((is_spam, spam_probability > 0.5)
            for _, is_spam, spam_probability in classified)

In [15]:
counts

Counter({(False, False): 704,
         (False, True): 33,
         (True, True): 101,
         (True, False): 38})

We count 101 true positives (spam classified as “spam”), 33 false positives (ham classified as “spam”), 704 true negatives (ham classified as “ham”), and 38 false negatives (spam classified as “ham”). <b>Precision</b> score is 

$$ \frac{\mathsf{true\;positives}}{\mathsf{true\;positives} + \mathsf{false\;positives}} = \frac{101} {101 + 33} = 75\%$$

and <b>recall</b> score is 

$$ \frac{\mathsf{true\;positives}}{\mathsf{true\;positives} + \mathsf{false\;negatives}} = \frac{101}{101 + 38} = 73\% \,,$$

really not bad for such a simple model.

### Most misclassified

We can sort messages by spam probability from smallest to largest using `sort`. The `row[2]` in sorting criterion refers to the probability component of a classified row of the form `[message, truth value, probability]`.

In [16]:
classified.sort(key=lambda row: row[2])
classified[:5]

[('Re[2]: Selling Wedded Bliss (was Re: Ouch...)',
  False,
  3.048747404869583e-10),
 ('Re[2]: Selling Wedded Bliss (was Re: Ouch...)',
  False,
  3.048747404869583e-10),
 ('Re[2]: Selling Wedded Bliss (was Re: Ouch...)',
  False,
  3.048747404869583e-10),
 ('Re[2]: Selling Wedded Bliss (was Re: Ouch...)',
  False,
  3.048747404869583e-10),
 ('Re: [Razor-users] Problem with Razor 2.14 and Spamassassin 2.41',
  False,
  6.209324406814169e-10)]

The highest predicted spam probabilities among the non-spam messages probabilities.

In [17]:
spammiest_hams = list(filter(lambda row: not row[1], classified))[-5:]
spammiest_hams

[('Attn programmers: support offered [FLOSS-Sarai Initiative]',
  False,
  0.9756129605140706),
 ('2000+ year old Greek computer reinterpreted', False, 0.9835355008104817),
 ('What to look for in your next smart phone (Tech Update)',
  False,
  0.9898719206903398),
 ('[ILUG-Social] Re: Important - reenactor insurance needed',
  False,
  0.9995349057803387),
 ('[ILUG-Social] Re: Important - reenactor insurance needed',
  False,
  0.9995349057803387)]

The lowest predicted spam probabilities among the actual spams.

In [18]:
hammiest_spams = list(filter(lambda row: row[1], classified))[:5]
hammiest_spams

[('Re: girls', True, 0.0009525186158413646),
 ('Introducing Chase Platinum for Students with a 0% Introductory APR',
  True,
  0.0012566691211122248),
 ('.Message report from your contact page....//ytu855 rkq',
  True,
  0.0015109358288642688),
 ('Testing a system, please delete', True, 0.0026920538836931215),
 ('Never pay for the goodz again (8SimUgQ)', True, 0.005911623221945951)]

### Spammiest and hammiest words

In [19]:
def p_spam_given_word(word_prob):
    """uses Bayes's theorem to compute Pr(spam | message contains word)"""
    # word_prob is one of the triplets produced by word_probabilities
    word, prob_if_spam, prob_if_not_spam = word_prob
    
    return prob_if_spam / (prob_if_spam + prob_if_not_spam)

words = sorted(classifier.word_probs, key=p_spam_given_word)

spammiest_words = words[-5:]
hammiest_words = words[:5]

In [20]:
spammiest_words

[('year', 0.028767123287671233, 0.00022893772893772894),
 ('rates', 0.031506849315068496, 0.00022893772893772894),
 ('sale', 0.031506849315068496, 0.00022893772893772894),
 ('systemworks', 0.036986301369863014, 0.00022893772893772894),
 ('money', 0.03972602739726028, 0.00022893772893772894)]

In [21]:
hammiest_words

[('spambayes', 0.0013698630136986301, 0.04601648351648352),
 ('users', 0.0013698630136986301, 0.036401098901098904),
 ('razor', 0.0013698630136986301, 0.030906593406593408),
 ('zzzzteana', 0.0013698630136986301, 0.029075091575091576),
 ('sadev', 0.0013698630136986301, 0.026785714285714284)]

Just to make it quick, the same model implemented here is contained in `scikit-learn` library as `BernoulliNB` model.